In [1]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [2]:
# Function to run optimization 
def run_optimization():
    try: 
        # Retrieve bounds and initial values from the GUI
        t_lower = float(t_lower_entry.get())
        t_upper = float(t_upper_entry.get())
        h_lower = float(h_lower_entry.get())
        h_upper = float(h_upper_entry.get())
        l_lower = float(l_lower_entry.get())
        l_upper = float(l_upper_entry.get())
        theta_lower = float(theta_lower_entry.get())
        theta_upper = float(theta_upper_entry.get())
        
        initial_t = float(initial_t_entry.get())
        initial_h = float(initial_h_entry.get())
        initial_l = float(initial_l_entry.get())
        initial_theta = float(initial_theta_entry.get())
        
        # Define the objective function
        def objective(x):
            t, h, l, theta = x
            return -70E3 * ((t / l) ** 3) * (np.cos(theta) / (((h / l) + np.sin(theta)) * np.sin(theta) ** 2))
        
        # Define the constraints
        def constraint1(x):
            t, h, l, theta = x
            return t - (h / 30)

        def constraint2(x):
            t, h, l, theta = x
            return t - (l / 10)

        def constraint3(x):
            t, h, l, theta = x
            value = 2700 * 1E-12 * ((2 + h / l) * t / l) / (2 * np.cos(theta) * ((h / l) + np.sin(theta))) * 100
            return value - 2E-8
        # Set bounds for variables
        bounds = [(t_lower, t_upper), (h_lower, h_upper), (l_lower, l_upper), (theta_lower, theta_upper)]

        # Initial guess
        x0 = [initial_t, initial_h, initial_l, initial_theta]

        # Define the constraints in the format required by scipy.optimize.minimize
        constraints = [{'type': 'ineq', 'fun': constraint1},
                       {'type': 'ineq', 'fun': constraint2},
                       {'type': 'ineq', 'fun': constraint3}]

        # Check if initial guess is feasible
        initial_constraints = [c['fun'](x0) for c in constraints]
        print(f'Initial guess: {x0}')
        print(f'Initial constraints values: {initial_constraints}')

        if any(c < 0 for c in initial_constraints):
            raise ValueError("Initial guess does not satisfy the constraints")

        # Solve the optimization problem using SLSQP method
        solution = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)

        # Print debugging information about the solution
        print(f'Solution: {solution}')
        print(f'Function value: {solution.fun}')
        print(f'Optimization status: {solution.status}')
        print(f'Message: {solution.message}')
        
        if not solution.success:
            raise ValueError("Optimization did not succeed: " + solution.message)
        
        # Extract the solution
        t, h, l, theta = solution.x
        
        # Plot the convex hull
        x_coords = [0, 0, l*np.cos(theta), 2*l*np.cos(theta), 2*l*np.cos(theta), l*np.cos(theta)]
        y_coords = [0, h, h+l*np.sin(theta), h, 0, -l*np.sin(theta)]
        P = np.vstack((x_coords, y_coords)).T

        # Display results
        result = f'Solution: {solution.x}\nObjective value: {-solution.fun}\nReason for termination: {solution.message}'
        messagebox.showinfo("Optimization Result", result)
        
        # Plot
        plt.figure()
        plt.plot(P[:,0], P[:,1], '*')
        for i in range(len(P)):
            plt.plot([P[i-1, 0], P[i, 0]], [P[i-1, 1], P[i, 1]], 'b-', linewidth=10*t)
        plt.fill(P[:,0], P[:,1], 'skyblue', alpha=0.5)
        plt.show()

    except Exception as e:
        messagebox.showerror("Error", str(e))


In [3]:

# Create the main window
root = tk.Tk()
root.title("Optimization Problem")

# Create and place the widgets
tk.Label(root, text="Lower Bound").grid(row=0, column=1)
tk.Label(root, text="Upper Bound").grid(row=0, column=2)
tk.Label(root, text="Initial Point").grid(row=0, column=3)

tk.Label(root, text="t").grid(row=1, column=0)
t_lower_entry = tk.Entry(root)
t_lower_entry.grid(row=1, column=1)
t_upper_entry = tk.Entry(root)
t_upper_entry.grid(row=1, column=2)
initial_t_entry = tk.Entry(root)
initial_t_entry.grid(row=1, column=3)

tk.Label(root, text="h").grid(row=2, column=0)
h_lower_entry = tk.Entry(root)
h_lower_entry.grid(row=2, column=1)
h_upper_entry = tk.Entry(root)
h_upper_entry.grid(row=2, column=2)
initial_h_entry = tk.Entry(root)
initial_h_entry.grid(row=2, column=3)

tk.Label(root, text="l").grid(row=3, column=0)
l_lower_entry = tk.Entry(root)
l_lower_entry.grid(row=3, column=1)
l_upper_entry = tk.Entry(root)
l_upper_entry.grid(row=3, column=2)
initial_l_entry = tk.Entry(root)
initial_l_entry.grid(row=3, column=3)

tk.Label(root, text="theta").grid(row=4, column=0)
theta_lower_entry = tk.Entry(root)
theta_lower_entry.grid(row=4, column=1)
theta_upper_entry = tk.Entry(root)
theta_upper_entry.grid(row=4, column=2)
initial_theta_entry = tk.Entry(root)
initial_theta_entry.grid(row=4, column=3)

tk.Button(root, text="Run Optimization", command=run_optimization).grid(row=5, column=0, columnspan=4)

# Default values
t_lower_entry.insert(0, "1")
t_upper_entry.insert(0, "10")
h_lower_entry.insert(0, "1")
h_upper_entry.insert(0, "50")
l_lower_entry.insert(0, "1")
l_upper_entry.insert(0, "50")
theta_lower_entry.insert(0, "0.01")
theta_upper_entry.insert(0, "1.6")

initial_t_entry.insert(0, "3")
initial_h_entry.insert(0, "25")
initial_l_entry.insert(0, "25")
initial_theta_entry.insert(0, "0.9")

# Start the GUI event loop
root.mainloop()



Initial guess: [3.0, 25.0, 25.0, 0.9]
Initial constraints values: [2.1666666666666665, 0.5, 2.384169496157144e-08]
Solution:  message: Inequality constraints incompatible
 success: False
  status: 4
     fun: -88454046.41965857
       x: [ 1.000e+01  2.346e+01  1.830e+01  1.000e-02]
     nit: 2
     jac: [-2.654e+07  3.741e+06  9.707e+06  1.776e+10]
    nfev: 10
    njev: 2
Function value: -88454046.41965857
Optimization status: 4
Message: Inequality constraints incompatible
Initial guess: [3.0, 25.0, 25.0, 0.9]
Initial constraints values: [2.1666666666666665, 0.5, 2.384169496157144e-08]
Solution:  message: Inequality constraints incompatible
 success: False
  status: 4
     fun: -88454046.41965857
       x: [ 1.000e+01  2.346e+01  1.830e+01  1.000e-02]
     nit: 2
     jac: [-2.654e+07  3.741e+06  9.707e+06  1.776e+10]
    nfev: 10
    njev: 2
Function value: -88454046.41965857
Optimization status: 4
Message: Inequality constraints incompatible
